## Feature Views
- Defined in `credit_feature_register.py`

### Entity: 
-  `SK_ID_CURR`


### Feature Views:
-  `static_feature_view`: customers demographic features, age, income, occupation, indicators and customer scores.
    - File: static_feature_table.parquet
    - Fields:
        - OCCUPATION_TYPE
        - AMT_INCOME_TOTAL
        - NAME_INCOME_TYPE
        - DAYS_LAST_PHONE_CHANGE
        - ORGANIZATION_TYPE
        - AMT_CREDIT
        - AMT_GOODS_PRICE
        - DAYS_REGISTRATION
        - AMT_ANNUITY
        - CODE_GENDER
        - DAYS_ID_PUBLISH
        - NAME_EDUCATION_TYPE
        - DAYS_EMPLOYED
        - DAYS_BIRTH
        - EXT_SOURCE_1
        - EXT_SOURCE_2
        - EXT_SOURCE_3
 


- `bureau_feature_view`: customer's credit info with other finanacial instuitues reported to Credit Bureau
    - File: bureau_feature_table.parquet
    - Fields: all fields



- `previous_loan_feature_view`: customer's previous credit card, cash loans, and repayment history.
    - File: previous_loan_features.parquet
    - Fields: all fields


In [34]:
!feast apply

/opt/anaconda3/envs/xy38/lib/python3.8/site-packages/feast/feature_view.py:100: DeprecationWarning: The argument 'input' is being deprecated. Please use 'batch_source' instead. Feast 0.13 and onwards will not support the argument 'input'.
  warnings.warn(
Created feature service credit_model_1
Deleted feature service credit_model

No changes to infrastructure


In [2]:
from datetime import datetime, timedelta
import pandas as pd
from datetime import date
from feast import FeatureStore


import pytz

In [3]:
store = FeatureStore(repo_path="../feature_repo/dev")


/opt/anaconda3/envs/xy38/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [38]:
entity_df = pd.DataFrame.from_dict(
    {
        "SK_ID_CURR": [100002, 100003, 100004],
        "label": [1, 0, 1], 
        "event_timestamp": [
            datetime(2022,2,24),
            datetime(2022,2,24),
            datetime(2022,2,24),
        ],
    }
)

In [9]:
## use feature view
# training_df = store.get_historical_features(
#     entity_df=entity_df,
#     features=[
#         "static_feature_view:EXT_SOURCE_1",
#         "static_feature_view:EXT_SOURCE_2",
#         "static_feature_view:EXT_SOURCE_3",
#         "bureau_feature_view:DEBT_CREDIT_RATIO",
#         "previous_loan_feature_view:AMT_BALANCE",
        
#     ],
# ).to_df()
# training_df

In [39]:
## use feature service
feature_service = store.get_feature_service("credit_model_1")
training_fs_df = store.get_historical_features(
    entity_df=entity_df,
    features=feature_service
).to_df()
training_fs_df

,event_timestamp,SK_ID_CURR,label,OCCUPATION_TYPE,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,DAYS_LAST_PHONE_CHANGE,ORGANIZATION_TYPE,AMT_CREDIT,AMT_GOODS_PRICE,...,CREDIT_TYPE_Loan for business development,CREDIT_TYPE_Loan for purchase of shares (margin lending),CREDIT_TYPE_Loan for the purchase of equipment,CREDIT_TYPE_Loan for working capital replenishment,CREDIT_TYPE_Microloan,CREDIT_TYPE_Mobile operator loan,CREDIT_TYPE_Mortgage,CREDIT_TYPE_Real estate loan,CREDIT_TYPE_Unknown type of loan,CREDIT_STATUS_EMA_AVG
0,2022-02-24 00:00:00+00:00,100002,1,Laborers,202500.0,Working,-1134.0,Business Entity Type 3,406597.5,351000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.183594
1,2022-02-24 00:00:00+00:00,100003,0,Core staff,270000.0,State servant,-828.0,School,1293502.5,1129500.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,2022-02-24 00:00:00+00:00,100004,1,Laborers,67500.0,Working,-815.0,Government,135000.0,135000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [7]:
!feast materialize-incremental 2022-02-25T00:00:00

/opt/anaconda3/envs/xy38/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Materializing 3 feature views to 2022-02-25 08:00:00+08:00 into the sqlite online store.

bureau_feature_view from 2021-11-27 01:36:48+08:00 to 2022-02-25 08:00:00+08:00:
100%|█████████████████████████████████████████████████████| 305811/305811 [04:49<00:00, 1054.94it/s]
static_feature_view from 2021-11-27 01:41:40+08:00 to 2022-02-25 08:00:00+08:00:
100%|█████████████████████████████████████████████████████| 307511/307511 [03:41<00:00, 1390.50it/s]
previous_loan_feature_view from 2021-11-27 01:45:25+08:00 to 2022-02-25 08:00:00+08:00:
100%|█████████████████████████████████████████████████████| 339587/339587 [01:31<00:00, 3692.53it/s]


### Get oneline feature

In [41]:
# features = [
#         "static_feature_view:EXT_SOURCE_1",
#         "static_feature_view:EXT_SOURCE_2",
#         "static_feature_view:EXT_SOURCE_3",
#         "bureau_feature_view:DEBT_CREDIT_RATIO",
#         "previous_loan_feature_view:AMT_BALANCE",
        
#     ],

# online_features = store.get_online_features(
#     features=features,
#     entity_rows=[
#         {"SK_ID_CURR": 100002},
#         {"SK_ID_CURR": 100003},
#     ]
# ).to_dict()

In [45]:
## use feature service
feature_service = store.get_feature_service("credit_model_1")
features = store.get_online_features(
    features=feature_service, entity_rows=[
        {"SK_ID_CURR": 100009},
        {"SK_ID_CURR": 100007},
    ]
).to_dict()
features

{'SK_ID_CURR': [100009, 100007],
 'OCCUPATION_TYPE': ['Accountants', 'Core staff'],
 'AMT_CREDIT': [1560726.0, 513000.0],
 'DAYS_EMPLOYED': [-3130, -3038],
 'EXT_SOURCE_2': [0.7239998579025269, 0.32273828983306885],
 'AMT_GOODS_PRICE': [1395000.0, 513000.0],
 'DAYS_BIRTH': [-13778, -19932],
 'AMT_ANNUITY': [41301.0, 21865.5],
 'DAYS_LAST_PHONE_CHANGE': [-1562.0, -1106.0],
 'EXT_SOURCE_3': [0.4920600950717926, None],
 'DAYS_REGISTRATION': [-1213.0, -4311.0],
 'ORGANIZATION_TYPE': ['Business Entity Type 3', 'Religion'],
 'EXT_SOURCE_1': [0.774761438369751, None],
 'AMT_INCOME_TOTAL': [171000.0, 121500.0],
 'DAYS_ID_PUBLISH': [-619, -3458],
 'NAME_INCOME_TYPE': ['Commercial associate', 'Working'],
 'NAME_EDUCATION_TYPE': ['Higher education', 'Secondary / secondary special'],
 'CODE_GENDER': ['F', 'M'],
 'CREDIT_TYPE_Interbank credit': [0.0, 0.0],
 'CREDIT_TYPE_Credit card': [2.0, 0.0],
 'CREDIT_TYPE_Loan for purchase of shares (margin lending)': [0.0, 0.0],
 'CREDIT_TYPE_Real estate loan'